In [ ]:
import os
# --- 1. 强制配置国内镜像 (最关键的一步) ---
# 必须在 import transformers 之前或者尽量靠前的位置设置
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

from transformers import BertModel

# --- 2. 路径配置 ---
# 确保这个路径和你报错信息里的路径一致
current_dir = os.getcwd()
target_path = os.path.abspath(os.path.join(current_dir, '../../data/pretrained_models/bert-base-chinese'))

print(f"🎯 目标修复路径: {target_path}")

# 检查目录是否存在，不存在则创建
if not os.path.exists(target_path):
    os.makedirs(target_path)

print("⏳ 正在尝试连接镜像站下载模型权重 (约 400MB)... 请耐心等待，不要中断")

try:
    # --- 3. 核心下载逻辑 ---
    # from_pretrained 会自动下载 pytorch_model.bin 或 model.safetensors
    # 我们先下载到内存/缓存
    model = BertModel.from_pretrained('bert-base-chinese')
    
    # --- 4. 保存到指定目录 ---
    print("✅ 下载完成，正在写入硬盘...")
    model.save_pretrained(target_path)
    
    print("-" * 30)
    print("🎉 修复成功！模型权重已保存。")
    print(f"📂 请检查目录: {target_path}")
    print("你应该能看到 'pytorch_model.bin' 或 'model.safetensors'")
    print("-" * 30)

except Exception as e:
    print("\n❌ 下载失败。请检查报错信息：")
    print(e)
    print("\n💡 建议：如果依然报网络错误，请检查是否开启了代理软件（有时代理反而会阻断镜像站）。")

# S2W6D3: 预训练模型与语义向量 (Pre-trained Models & Embeddings)

如果说前两天我们还在处理“皮毛”（把文本变成整数 ID），那么今天我们要正式触及 Transformer 的**灵魂**。

我们要加载那个 400MB 的大脑（权重），让计算机第一次真正“**理解**”语义。我们将解决一个核心问题：**机器如何知道“拿个苹果”和“去取水果”是相似的指令？**

**🎯 核心目标**:

1.  **加载权重**: 学会使用 `BertModel` 加载预训练好的参数。
2.  **理解输出**: 彻底分清 `last_hidden_state` 和 `pooler_output` 的区别（面试必考）。
3.  **语义相似度**: 利用 Embedding 向量计算余弦相似度，实现具身智能中的**指令模糊匹配**。

## 1 🛠️ 环境确认 (Asset Check)

在开始之前，确保你的 `data/` 目录下已经有模型文件。为了保证代码能跑，我们先执行一段\*\*“资产加载”\*\*代码。

In [ ]:
import os
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel

# 1. 定位资产路径 (指向 data/pretrained_models/bert-base-chinese)
current_dir = os.getcwd()
model_path = os.path.abspath(os.path.join(current_dir, '../../data/pretrained_models/bert-base-chinese'))

# 2. 检查模型是否存在
if not os.path.exists(os.path.join(model_path, 'pytorch_model.bin')) and \
   not os.path.exists(os.path.join(model_path, 'model.safetensors')):
    print(f"⚠️ 警告: 在 {model_path} 下未找到模型权重文件。")
    print("请先运行 Day 2 的下载脚本，或者检查网络连接。")
else:
    print(f"✅ 模型资产就绪: {model_path}")

# 3. 加载 Tokenizer 和 Model
# 注意: 这里全部使用本地路径，断网可用
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertModel.from_pretrained(model_path)
model.eval() # 必加！切换到评估模式

print("🚀 BERT 模型加载成功！大脑已上线。")

✅ 模型资产就绪: /home/goodminton/study/AI-Interview-Sprint/data/pretrained_models/bert-base-chinese
🚀 BERT 模型加载成功！大脑已上线。


## 2 🧠 核心理论：BERT 的两种输出 (The Dual Outputs)

当你把`input_ids`喂给BERT后，它会吐出两个主要的东西。这是**最容易混淆**的概念，也是**面试高频坑点**。

### 2.1 `last_hidden_state`（序列输出）

- **形状**：`[Batch, Seq_Len, 768]`
- **含义**：**每个Token**的语义向量。
- **用途**：
    - **NER（命名实体识别）**：需要判断每个字是不是人文/地文。
    - **QA（问答）**：需要从文章中找到答案的起始和结束位置。

### 2.2 `pooler_output`（池化输出）

- **形状**：`[Batch, 768]`
- **含义**：代表 **整个句子** 的语义向量。
- **原理**: 它提取了 `[CLS]` 位置的向量，并经过了一个线性层 (`Linear` + `Tanh`) 进一步处理。
- **用途**：**文本分类**、**相似度计算**。

## 3 💻 代码实战：提取向量 (Feature Extraction)

我们来看看这两个输出到底长什么样。

In [ ]:
# 1. 准备数据
text = "具身智能是未来的方向"
inputs = tokenizer(text, return_tensors="pt")

# 2. 前向传播 (No Gradients needed for inference)
with torch.no_grad():
    outputs = model(**inputs)

# 3. 获取两种输出
last_hidden_state = outputs.last_hidden_state
pooler_output = outputs.pooler_output

print(f"输入文本: {text}")
print("-" * 30)
print(f"1. last_hidden_state Shape: {last_hidden_state.shape}")
# 预期: [1, 12, 768] (1句话, 12个token, 768维)

print(f"2. pooler_output Shape:     {pooler_output.shape}")
# 预期: [1, 768] (1句话, 768维)

输入文本：具身智能是未来的方向
------------------------------
1. last_hidden_state Shape:torch.Size([1, 12, 768])
2. pooler_output Shape: torch.Size([1, 768])


## 4 🤖 具身实战：指令相似度匹配 (Semantic Similarity)

这是你作为机器人负责人的**核心技能**。
用户说话是很随意的，但机器人的 API 是固定的。我们需要把用户的“自然语言”映射到“标准指令”。

  * **用户说**: "把那个水杯拿给我" / "取一下杯子" / "Grab the cup"
  * **机器人 API**: `pick_up_object(target='cup')`

**原理**: 计算两个句子向量的 **余弦相似度 (Cosine Similarity)**。

In [13]:
def get_embedding(text):
    """辅助函数：输入文本，返回句向量 (pooler_output)"""
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.pooler_output

# --- 模拟场景 ---
standard_cmd = "抓取面前的物体"  # 标准指令
user_cmd_1 = "帮我把东西拿起来"  # 意思相近
user_cmd_2 = "向左旋转90度"      # 意思完全不同

# 1. 提取向量
vec_std = get_embedding(standard_cmd)
vec_1 = get_embedding(user_cmd_1)
vec_2 = get_embedding(user_cmd_2)

# 2. 计算余弦相似度
# Cosine Sim = (A . B) / (|A| * |B|)
score_1 = F.cosine_similarity(vec_std, vec_1)
score_2 = F.cosine_similarity(vec_std, vec_2)

print(f"基准指令: {standard_cmd}")
print("-" * 30)
print(f"指令1: '{user_cmd_1}' | 相似度: {score_1.item():.4f}")
print(f"指令2: '{user_cmd_2}' | 相似度: {score_2.item():.4f}")

# --- 判定逻辑 ---
threshold = 0.85 # 阈值
if score_1 > threshold:
    print(">> 判定: 执行抓取动作")
else:
    print(">> 判定: 指令不匹配")

基准指令: 抓取面前的物体
------------------------------
指令1: '帮我把东西拿起来' | 相似度: 0.9101
指令2: '向左旋转90度' | 相似度: 0.8965
>> 判定: 执行抓取动作


**实验预期**: `score_1` 应该显著高于 `score_2`。你会发现 BERT 真的很懂中文！

## 5 💥 面试深挖：CLS 还是 Mean Pooling?

面试官可能会问：**“直接用BERT 的 `pooler_output` 做相似度计算是最好的吗？”**

  * **回答**: 其实不是。
  * **原因**: 原生BERT的`pooler_output`是为了NSP (Next Sentence Prediction)任务训练的，并未针对“语义相似度”做优化。
  * **更优解**: **Mean Pooling (平均池化)**。即把 `last_hidden_state` 中所有 Token 的向量取平均值。这通常比直接用 `[CLS]` 效果更好。
  * **SOTA 解法**: 使用专门训练过的 **Sentence-BERT (SBERT)** 模型。

*(注：今天的代码用 pooler\_output 演示是可以的，但在 Stage 3 做 RAG 时我们会换更高级的方法)*


## 6\. ⚔️ 每日算法题 (LeetCode Daily)

既然今天讲的是**字符串匹配**的升级版（语义匹配），那我们来回顾一下经典的**字符串查找**算法。

### 🎯 题目: [LeetCode 28](../../LeetCode%20practice/1-50.ipynb). 找出字符串中第一个匹配项的下标 (Find the Index of the First Occurrence in a String)

  * **难度**: Easy (但要写好不容易)
  * **描述**: 实现 `strStr()` 函数。在 `haystack` 中找出 `needle` 第一个出现的位置。
  * **关联**:
      * **传统搜索**: 必须字面上完全匹配 (Exact Match)。这道题就是传统搜索的极致。
      * **BERT 搜索**: 语义匹配 (Semantic Match)。你可以对比一下这两种搜索方式的差异。
  * **要求**: 哪怕可以用 python 的 `.find()` 或 `.index()` 秒杀，也请尝试理解 **滑动窗口** 或 **KMP 算法** 的思想（KMP 面试不强求手写，但要知道原理）。


### ✅ Day 3 任务清单

1.  **运行代码**: 成功加载本地模型，不报错。
2.  **验证相似度**: 运行具身实战代码，看看 BERT 认为 "拿东西" 和 "向左转" 的相似度分别是多少。
3.  **LeetCode**: 完成 LeetCode 28。